In [37]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
s = pd.read_csv("season.csv")

In [3]:
final = s[["cust_nbr", "fisc_wk_id", "ttl_cases"]]

In [4]:
final.head()

,cust_nbr,fisc_wk_id,ttl_cases
0,5785,202051,35.000
1,11866,202051,1.000
2,36111,202051,138.248
3,36475,202051,282.498
4,104505,202051,17.000


In [5]:
all_wk = list(range(202051, 202053)) + list(range(202101, 202154)) + list(range(202201, 202253)) + list(range(202301, 202350))

In [6]:
customers = final["cust_nbr"].unique()

In [7]:
all_data = []

for c in customers:
    all_data.append([c]*len(all_wk))

In [8]:
all_d = pd.DataFrame()

In [9]:
all_d["cust_nbr"] = np.array(all_data).flatten()

In [10]:
all_d["fisc_wk_id"] = all_wk * len(customers)

In [11]:
full_data = all_d.merge(final, how = "left", on =["cust_nbr", "fisc_wk_id"] )

In [12]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 501228 entries, 0 to 501227
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   cust_nbr    501228 non-null  int64  
 1   fisc_wk_id  501228 non-null  int64  
 2   ttl_cases   157467 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 15.3 MB


In [13]:
full_data

,cust_nbr,fisc_wk_id,ttl_cases
0,5785,202051,35.0
1,5785,202052,32.0
2,5785,202101,41.5
3,5785,202102,34.0
4,5785,202103,33.0
...,...,...,...
501223,672453,202345,NaN
501224,672453,202346,NaN
501225,672453,202347,NaN
501226,672453,202348,NaN


In [14]:
full_data.fillna(0, inplace = True)

In [15]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 501228 entries, 0 to 501227
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   cust_nbr    501228 non-null  int64  
 1   fisc_wk_id  501228 non-null  int64  
 2   ttl_cases   501228 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 15.3 MB


In [16]:
full_data.sort_values("fisc_wk_id", inplace = True)

In [17]:
full_data.reset_index(inplace = True, drop = True)

In [18]:
full_data["year"] = full_data["fisc_wk_id"].astype(str).str[:4].astype(int)

In [19]:
full_data

,cust_nbr,fisc_wk_id,ttl_cases,year
0,5785,202051,35.000,2020
1,490748,202051,0.000,2020
2,252040,202051,170.916,2020
3,671882,202051,0.000,2020
4,513960,202051,0.000,2020
...,...,...,...,...
501223,663979,202349,0.000,2023
501224,670117,202349,0.000,2023
501225,663995,202349,21.000,2023
501226,670201,202349,0.000,2023


In [20]:
grouped = full_data.groupby(["cust_nbr","year"])["ttl_cases"].agg(["mean",lambda x: np.percentile(x,20)]).reset_index()

In [21]:
grouped

,cust_nbr,year,mean,<lambda_0>
0,5785,2020,33.500000,32.6
1,5785,2021,32.812755,30.0
2,5785,2022,29.592885,22.0
3,5785,2023,31.338735,26.6
4,11866,2020,0.500000,0.2
...,...,...,...,...
12847,936971,2023,0.000000,0.0
12848,952754,2020,19.500000,19.2
12849,952754,2021,20.415094,9.6
12850,952754,2022,16.950308,2.6


In [22]:
grouped.columns = ['cust_nbr', 'year', "year avg cases","min_20_percentile"]

In [23]:
grouped

,cust_nbr,year,year avg cases,min_20_percentile
0,5785,2020,33.500000,32.6
1,5785,2021,32.812755,30.0
2,5785,2022,29.592885,22.0
3,5785,2023,31.338735,26.6
4,11866,2020,0.500000,0.2
...,...,...,...,...
12847,936971,2023,0.000000,0.0
12848,952754,2020,19.500000,19.2
12849,952754,2021,20.415094,9.6
12850,952754,2022,16.950308,2.6


In [24]:
full_data = full_data.merge(grouped, how = "left", left_on = ["cust_nbr", "year",], right_on =["cust_nbr","year",])

In [25]:
full_data

,cust_nbr,fisc_wk_id,ttl_cases,year,year avg cases,min_20_percentile
0,5785,202051,35.000,2020,33.500000,32.600
1,490748,202051,0.000,2020,0.000000,0.000
2,252040,202051,170.916,2020,150.916000,138.916
3,671882,202051,0.000,2020,0.000000,0.000
4,513960,202051,0.000,2020,0.000000,0.000
...,...,...,...,...,...,...
501223,663979,202349,0.000,2023,15.698102,0.000
501224,670117,202349,0.000,2023,8.626163,0.000
501225,663995,202349,21.000,2023,19.593082,0.000
501226,670201,202349,0.000,2023,0.000000,0.000


In [26]:
def lth_20(row):
    if  row["ttl_cases"] <= row["min_20_percentile"]:
        return 0
    return 1

In [27]:
full_data["<= 20th percentile"] = full_data.apply(lth_20, axis = 1)

In [28]:
full_data.head(15)

,cust_nbr,fisc_wk_id,ttl_cases,year,year avg cases,min_20_percentile,<= 20th percentile
0,5785,202051,35.000,2020,33.500,32.600,1
1,490748,202051,0.000,2020,0.000,0.000,0
2,252040,202051,170.916,2020,150.916,138.916,1
3,671882,202051,0.000,2020,0.000,0.000,0
4,513960,202051,0.000,2020,0.000,0.000,0
5,422402,202051,0.000,2020,0.000,0.000,0
6,494807,202051,0.000,2020,0.000,0.000,0
7,671889,202051,0.000,2020,0.000,0.000,0
8,668835,202051,0.000,2020,0.000,0.000,0
9,496406,202051,0.000,2020,0.000,0.000,0


In [29]:
full_data.to_csv("20th Percentile.csv", index = False)

In [41]:
per = pd.read_csv("20th Percentile.csv")

In [42]:
per.head()

,cust_nbr,fisc_wk_id,ttl_cases,year,year avg cases,min_20_percentile,<= 20th percentile
0,5785,202051,35.000,2020,33.500,32.600,1
1,490748,202051,0.000,2020,0.000,0.000,0
2,252040,202051,170.916,2020,150.916,138.916,1
3,671882,202051,0.000,2020,0.000,0.000,0
4,513960,202051,0.000,2020,0.000,0.000,0


In [43]:
per["fisc_wk"] = per["fisc_wk_id"].astype(str).str[-2:].astype(int)

In [44]:
train = per[["cust_nbr", "fisc_wk", "<= 20th percentile"]]

In [45]:
train

,cust_nbr,fisc_wk,<= 20th percentile
0,5785,51,1
1,490748,51,0
2,252040,51,1
3,671882,51,0
4,513960,51,0
...,...,...,...
501223,663979,49,0
501224,670117,49,0
501225,663995,49,1
501226,670201,49,0


In [46]:
X = train[['cust_nbr', 'fisc_wk']]
y = train['<= 20th percentile'] 

In [47]:
model = LogisticRegression()
model.fit(X, y)

LogisticRegression()

In [48]:
y_pred = model.predict(X)

In [55]:
train["probablity"] = model.predict_proba(X)[:,1]

C:\Users\Waqas\AppData\Local\Temp\ipykernel_6924\3586513952.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["probablity"] = model.predict_proba(X)[:,1]


In [56]:
train

,cust_nbr,fisc_wk,<= 20th percentile,probablity
0,5785,51,1,0.497622
1,490748,51,0,0.308551
2,252040,51,1,0.397855
3,671882,51,0,0.248854
4,513960,51,0,0.300468
...,...,...,...,...
501223,663979,49,0,0.251291
501224,670117,49,0,0.249397
501225,663995,49,1,0.251286
501226,670201,49,0,0.249371


In [59]:
from sklearn.preprocessing import PolynomialFeatures

In [57]:
X = train[['cust_nbr', 'fisc_wk']]  
y = train['<= 20th percentile']  

In [60]:
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X)

In [61]:
model = LogisticRegression(solver='liblinear', penalty='l1', C=1.0)
model.fit(X_train_poly, y)

C:\Users\Waqas\anaconda3\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LogisticRegression(penalty='l1', solver='liblinear')

In [62]:
y_pred = model.predict(X_train_poly)

In [64]:
train["probablity"] = model.predict_proba(X_train_poly)[:,1]

C:\Users\Waqas\AppData\Local\Temp\ipykernel_6924\2869314530.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["probablity"] = model.predict_proba(X_train_poly)[:,1]


In [66]:
train.to_csv("probability.csv", index = False)